In [1]:
# ==============================================================================
# VÉGLEGES PYTHON KÓD (tx_cost MEZŐVEL)
# ==============================================================================
import pandas as pd
import yfinance as yf
import pandas_datareader as pdr
import os
from datetime import datetime
from kaggle_secrets import UserSecretsClient
from pandas.tseries.offsets import MonthEnd

def get_metadata():
    """Visszaadja a projektekhez szükséges összes pénzügyi faktor metaadatait, a tx_cost-ot is beleértve."""
    meta_df = pd.DataFrame([
        # --- ETFS (Yahoo Finance) ---
        {"symbol": "BIL", "name": "USD Cash", "asset_class": "TREASURY", "source": "yahoo", "type": "asset_return", "tx_cost": 0.0030},
        {"symbol": "SPY", "name": "US Large Cap", "asset_class": "EQUITY", "source": "yahoo", "type": "asset_return", "tx_cost": 0.0075},
        {"symbol": "IWM", "name": "US Small Cap", "asset_class": "EQUITY", "source": "yahoo", "type": "asset_return", "tx_cost": 0.0075},
        {"symbol": "VNQ", "name": "US REIT", "asset_class": "REAL_ASSET", "source": "yahoo", "type": "asset_return", "tx_cost": 0.0075},
        {"symbol": "GLD", "name": "Gold", "asset_class": "REAL_ASSET", "source": "yahoo", "type": "asset_return", "tx_cost": 0.0075},
        {"symbol": "DBA", "name": "Agriculture", "asset_class": "REAL_ASSET", "source": "yahoo", "type": "asset_return", "tx_cost": 0.0075},
        {"symbol": "SHY", "name": "Short Gov", "asset_class": "TREASURY", "source": "yahoo", "type": "asset_return", "tx_cost": 0.0030},
        {"symbol": "IEF", "name": "Mid Gov", "asset_class": "TREASURY", "source": "yahoo", "type": "asset_return", "tx_cost": 0.0030},
        {"symbol": "TLT", "name": "Long Gov", "asset_class": "TREASURY", "source": "yahoo", "type": "asset_return", "tx_cost": 0.0030},
        {"symbol": "LQD", "name": "IG Corp", "asset_class": "CREDIT", "source": "yahoo", "type": "asset_return", "tx_cost": 0.0075},
        {"symbol": "HYG", "name": "HY Corp", "asset_class": "CREDIT", "source": "yahoo", "type": "asset_return", "tx_cost": 0.0075},
        
        # --- MAKRO (Yahoo Finance) ---
        {"symbol": "^VIX", "name": "VIX Index", "asset_class": "MACRO", "source": "yahoo", "type": "macro_level", "tx_cost": pd.NA},

        # --- MAKRO (FRED) ---
        {"symbol": "T10Y2Y", "name": "Yield Curve (10Y-2Y)", "asset_class": "MACRO", "source": "fred", "type": "macro_level", "tx_cost": pd.NA},
        {"symbol": "CPIAUCSL", "name": "CPI YoY", "asset_class": "MACRO", "source": "fred", "type": "macro_cpi_yoy", "tx_cost": pd.NA},
    ])
    return meta_df

def create_financial_factor_dataset(meta_df: pd.DataFrame, 
                                    start_date: str = "1999-12-31",
                                    output_path: str = "financial_factors.csv.gz",
                                    meta_output_path: str = "financial_factors_meta.csv.gz") -> pd.DataFrame:
    print(">>> Faktor adatbázis készítése PYTHON használatával...")
    
    today = pd.to_datetime('today').normalize()
    end_date = today.replace(day=1) - pd.Timedelta(days=1)
    end_date_str = end_date.strftime('%Y-%m-%d')
    print(f"Adatok letöltése a '{start_date}' és '{end_date_str}' közötti időszakra.")

    processed_series = []

    yahoo_symbols = meta_df[meta_df['source'] == 'yahoo']['symbol'].tolist()
    if yahoo_symbols:
        yf_data = yf.download(yahoo_symbols, start=start_date, end=end_date_str, progress=False, auto_adjust=True)
        asset_symbols = meta_df[(meta_df['source'] == 'yahoo') & (meta_df['type'] == 'asset_return')]['symbol'].tolist()
        if asset_symbols:
            monthly_prices = yf_data['Close'][asset_symbols].resample('ME').last()
            returns = monthly_prices.pct_change().iloc[1:]
            processed_series.append(returns)
        macro_symbols = meta_df[(meta_df['source'] == 'yahoo') & (meta_df['type'] == 'macro_level')]['symbol'].tolist()
        if macro_symbols:
            vix_monthly_avg = yf_data['Close']['^VIX'].resample('ME').mean().to_frame(name='^VIX')
            processed_series.append(vix_monthly_avg)

    fred_symbols = meta_df[meta_df['source'] == 'fred']['symbol'].tolist()
    if fred_symbols:
        try:
            user_secrets = UserSecretsClient()
            os.environ['FRED_API_KEY'] = user_secrets.get_secret("FRED_API_KEY")
        except: print("FIGYELEM: FRED_API_KEY nem található.")
        fred_data = pdr.get_data_fred(fred_symbols, start=start_date, end=end_date_str)
        
        for symbol, ftype in meta_df[meta_df['source'] == 'fred'][['symbol', 'type']].values:
            if ftype == 'macro_cpi_yoy':
                monthly_last = fred_data[symbol].resample('ME').last()
                yoy_change = monthly_last.pct_change(12, fill_method=None) * 100
                fred_data[symbol] = yoy_change
            else:
                fred_data[symbol] = fred_data[symbol].resample('ME').last()
        processed_series.append(fred_data.resample('ME').last())

    combined_df = pd.concat(processed_series, axis=1)
    combined_df.ffill(inplace=True)
    final_df = combined_df.dropna()
    final_df.index.name = 'Date'
    final_df.index = final_df.index + MonthEnd(0)
    
    final_df.to_csv(output_path, compression='gzip')
    print(f"\nAdatok elmentve ide: {output_path}")
    meta_df.to_csv(meta_output_path, compression='gzip', index=False)
    print(f"Metaadatok elmentve ide: {meta_output_path}")
    print(f"\nKész! Végső adatsor mérete: {final_df.shape[0]} sor, {final_df.shape[1]} oszlop.")
    return final_df

# === Futtatás ===
META_DATA_PY = get_metadata()
final_data_py = create_financial_factor_dataset(META_DATA_PY, start_date="1999-12-31")
print("\nPython által generált DataFrame fejléce és lábléce:")
print(final_data_py.head())
print(final_data_py.tail())

>>> Faktor adatbázis készítése PYTHON használatával...
Adatok letöltése a '1999-12-31' és '2025-09-30' közötti időszakra.

Adatok elmentve ide: financial_factors.csv.gz
Metaadatok elmentve ide: financial_factors_meta.csv.gz

Kész! Végső adatsor mérete: 220 sor, 14 oszlop.

Python által generált DataFrame fejléce és lábléce:
                 BIL       SPY       IWM       VNQ       GLD       DBA  \
Date                                                                     
2007-06-30  0.004367 -0.014621 -0.014383 -0.090078 -0.019378  0.001897   
2007-07-31  0.003996 -0.031310 -0.069964 -0.082593  0.023650 -0.007197   
2007-08-31  0.004258  0.012833  0.021271  0.067445  0.011096  0.016406   
2007-09-30  0.001899  0.038714  0.018984  0.039173  0.105081  0.107357   
2007-10-31  0.003935  0.013567  0.028486  0.020991  0.069514 -0.007797   

                 SHY       IEF       TLT       LQD       HYG       ^VIX  \
Date                                                                      
2007-

In [2]:
print(final_data_py.info())
print(final_data_py.describe())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 220 entries, 2007-06-30 to 2025-09-30
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   BIL       220 non-null    float64
 1   SPY       220 non-null    float64
 2   IWM       220 non-null    float64
 3   VNQ       220 non-null    float64
 4   GLD       220 non-null    float64
 5   DBA       220 non-null    float64
 6   SHY       220 non-null    float64
 7   IEF       220 non-null    float64
 8   TLT       220 non-null    float64
 9   LQD       220 non-null    float64
 10  HYG       220 non-null    float64
 11  ^VIX      220 non-null    float64
 12  T10Y2Y    220 non-null    float64
 13  CPIAUCSL  220 non-null    float64
dtypes: float64(14)
memory usage: 25.8 KB
None
              BIL         SPY         IWM         VNQ         GLD         DBA  \
count  220.000000  220.000000  220.000000  220.000000  220.000000  220.000000   
mean     0.001052    0.009288    0.007757  